**Investigate Performance Difference Between CellSeg and Matterport**

In [1]:
import sys, os
mrcnn_path='../Mask_RCNN-TF2'
assert os.path.exists(mrcnn_path), 'mrcnn_path does not exist: '+mrcnn_path
sys.path.insert(0, "../CellSeg/src") # to import CellSeg
sys.path.insert(0, mrcnn_path) 

%matplotlib inline
import matplotlib.pyplot as plt

import os, sys, datetime, glob,pdb
import numpy as np
import skimage.io
from imgaug import augmenters as iaa
from skimage import img_as_ubyte, img_as_uint

import syotil

# Mask_RCNN-TF2 matterport alsombra
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import model as modellib
from mrcnn import visualize

# CellSeg
from cvmodelconfig import CVSegmentationConfig

# dense_cell_seg
from tfmrcnn_CellsegDataset import *
from tfmrcnn_StringerConfig import *

In [2]:
basedir = './' 
MODELS_DIR = os.path.join(basedir, "models")
if not os.path.exists(MODELS_DIR):
    os.makedirs(MODELS_DIR)

In [3]:
def mask_3dto2d(mask, scores):
    "transform a mask array that is [H, W, count] to [H, W]"
    assert mask.ndim == 3, "Mask must be [H, W, count]"
    # If mask is empty, return line with image ID only
    if mask.shape[-1] == 0:
        return np.zeros(mask.shape[:3])
    # Remove mask overlaps
    # Multiply each instance mask by its score order
    # then take the maximum across the last dimension
    order = np.argsort(scores)[::-1] + 1  # 1-based descending
    mask = np.max(mask * np.reshape(order, [1, 1, -1]), -1)
    return mask

def remove_overlaps(masks, cellpix, medians):
    """ replace overlapping mask pixels with mask id of closest mask
        masks = Nmasks x Ly x Lx
    """
    overlaps = np.array(np.nonzero(cellpix>1.5)).T
    dists = ((overlaps[:,:,np.newaxis] - medians.T)**2).sum(axis=1)
    tocell = np.argmin(dists, axis=1)
    masks[:, overlaps[:,0], overlaps[:,1]] = 0
    masks[tocell, overlaps[:,0], overlaps[:,1]] = 1

    # labels should be 1 to mask.shape[0]
    masks = masks.astype(int) * np.arange(1,masks.shape[0]+1,1,int)[:,np.newaxis,np.newaxis]
    masks = masks.sum(axis=0)
    return masks

In [4]:
import argparse
parser = argparse.ArgumentParser(description='Mask R-CNN for cell counting and segmentation')
args = argparse.Namespace(gpu_id=1, results_dir="images/test_tfmrcnn_cellseg2", dataset="images/test_images", batch_size=2, weights_path="../CellSeg/src/modelFiles/final_weights.h5")
os.environ["CUDA_VISIBLE_DEVICES"]=str(args.gpu_id)

print("Running on: {}".format(args.dataset))
    
if args.results_dir: 
    results_dir=args.results_dir
else:
    results_dir = "testmasks_{:%Y%m%dT%H%M%S}".format(datetime.datetime.now())
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
print("Results saved to: {}".format(args.results_dir))

print("Weights: {}".format(args.weights_path))

Running on: images/test_images
Results saved to: images/test_tfmrcnn_cellseg2
Weights: ../CellSeg/src/modelFiles/final_weights.h5


In [5]:
# Read dataset
dataset = CellsegDataset()
dataset.load_data(args.dataset, '')
dataset.prepare()

**There is a huge difference in performance using different configs as the next two cells show.**

In [7]:
config1 = StringerEvalConfig()
config1.NAME = "cellpose"

model = modellib.MaskRCNN(mode="inference", config=config1, model_dir=MODELS_DIR)    
model.load_weights(args.weights_path, by_name=True)

remove_overlap=True# masks with overlap removed work better
AP_arr=[]
for image_id in dataset.image_ids:    
    image = dataset.load_image(image_id)
    r = model.detect([image], verbose=0)[0]
    mask = r["masks"]
    if remove_overlap:
        medians = []
        for m in range(mask.shape[-1]):
            ypix, xpix = np.nonzero(mask[:,:,m])
            medians.append(np.array([ypix.mean(), xpix.mean()]))
        mask = np.int32(remove_overlaps(np.transpose(mask, (2,0,1)), mask.sum(axis=-1), np.array(medians)))             
    else:
        mask = mask_3dto2d(mask, r["scores"])
#     skimage.io.imsave("{}/{}.png".format(results_dir, dataset.image_info[image_id]["id"].replace("_img","_masks")), 
#                       img_as_uint(mask), check_contrast=False)

    truth=skimage.io.imread("images/test_gtmasks/"+dataset.image_info[image_id]["id"].replace("img","masks")+".png")
    AP_arr.append(syotil.csi(mask, truth))# masks may lost one pixel
    
print(AP_arr)
print("mAP: "+str(np.mean(AP_arr)))

Instructions for updating:
Use fn_output_signature instead


/app/software/fhPython/3.7.12-GCCcore-11.2.0/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


[0.3166023166023166, 0.25667351129363447, 0.2857142857142857, 0.3076923076923077, 0.29672447013487474, 0.24761904761904763, 0.2558139534883721]
mAP: 0.2809771275064056


In [8]:
# CellSeg configuration
config2 = CVSegmentationConfig(smallest_side=256)
config2.NAME = "CellSeg"
config2.PRE_NMS_LIMIT = 6000

model = modellib.MaskRCNN(mode="inference", config=config2, model_dir=MODELS_DIR)    
model.load_weights(args.weights_path, by_name=True)

remove_overlap=True # masks with overlap removed work better
AP_arr=[]
for image_id in dataset.image_ids:    
    image = dataset.load_image(image_id)
    r = model.detect([image], verbose=0)[0]
    mask = r["masks"]
    if remove_overlap:
        medians = []
        for m in range(mask.shape[-1]):
            ypix, xpix = np.nonzero(mask[:,:,m])
            medians.append(np.array([ypix.mean(), xpix.mean()]))
        mask = np.int32(remove_overlaps(np.transpose(mask, (2,0,1)), mask.sum(axis=-1), np.array(medians)))             
    else:
        mask = mask_3dto2d(mask, r["scores"])
#     skimage.io.imsave("{}/{}.png".format(results_dir, dataset.image_info[image_id]["id"].replace("_img","_masks")), 
#                       img_as_uint(mask), check_contrast=False)
    truth=skimage.io.imread("images/test_gtmasks/"+dataset.image_info[image_id]["id"].replace("img","masks")+".png")
    AP_arr.append(syotil.csi(mask, truth))# masks may lost one pixel
    
print(AP_arr)
print("mAP: "+str(np.mean(AP_arr)))

[0.3860294117647059, 0.3617886178861789, 0.4020100502512563, 0.3314285714285714, 0.42401500938086306, 0.4156479217603912, 0.4040632054176072]
mAP: 0.38928325541279624


In [14]:
config3.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
CPU_COUNT                      0
DETECTION_MAX_INSTANCES        400
DETECTION_MIN_CONFIDENCE       0
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                2.0
IMAGE_RESIZE_MODE              pad64
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE       

In [16]:
config3 = StringerEvalConfig()
config3.NAME = "cellpose"
# 0.2809771275064056

# following are the modifications based on config2 (CellSeg)

config3.BACKBONE                       = "resnet101" # changed from resnet50
# 0.3445496033869464 

config3.MEAN_PIXEL                     = [123.7, 116.8, 103.9] # changed from [43.53 39.56 48.22]
# 0.38641245939340807 

config3.DETECTION_MIN_CONFIDENCE       = 0.7 # changed from 0
# 0.4101849700393826 

# these do not make material differences
config3.DETECTION_MAX_INSTANCES        = 2000
config3.POST_NMS_ROIS_INFERENCE        = 6000
config3.POST_NMS_ROIS_TRAINING         = 2000
config3.RPN_NMS_THRESHOLD              = 0.8
config3.RPN_TRAIN_ANCHORS_PER_IMAGE    = 256
config3.TRAIN_ROIS_PER_IMAGE           = 200
config3.ASPECT_RATIO                   = 1
config3.MIN_ENLARGE                    = 1
config3.ZOOM                           = False
config3.IMAGE_SHAPE                    = [0, 0, 3]
# 0.41208502417927967

# config3.IMAGE_MIN_SCALE                = False
# # 0.29976782879444375
# # It is 2 by default. CellSeg probably have it as False in the config, but change it in the code


model = modellib.MaskRCNN(mode="inference", config=config3, model_dir=MODELS_DIR)    
model.load_weights(args.weights_path, by_name=True)

remove_overlap=True# masks with overlap removed work better
AP_arr=[]
for image_id in dataset.image_ids:    
    image = dataset.load_image(image_id)
    r = model.detect([image], verbose=0)[0]
    mask = r["masks"]
    if remove_overlap:
        medians = []
        for m in range(mask.shape[-1]):
            ypix, xpix = np.nonzero(mask[:,:,m])
            medians.append(np.array([ypix.mean(), xpix.mean()]))
        mask = np.int32(remove_overlaps(np.transpose(mask, (2,0,1)), mask.sum(axis=-1), np.array(medians)))             
    else:
        mask = mask_3dto2d(mask, r["scores"])
        
#     skimage.io.imsave("{}/{}.png".format(results_dir, dataset.image_info[image_id]["id"].replace("_img","_masks")), 
#                       img_as_uint(mask), check_contrast=False)

    truth=skimage.io.imread("images/test_gtmasks/"+dataset.image_info[image_id]["id"].replace("img","masks")+".png")
    AP_arr.append(syotil.csi(mask, truth))# masks may lost one pixel
    
print(AP_arr)
print("mAP: "+str(np.mean(AP_arr)))

[0.3786008230452675, 0.32, 0.4489795918367347, 0.3409090909090909, 0.39787234042553193, 0.36049382716049383, 0.34438775510204084]
mAP: 0.3701776326398799
